In [43]:
import pandas as pd
import numpy as np

In [57]:
df = pd.read_csv('All orders - short custom description.csv', low_memory=False)
df.drop_duplicates(inplace=True)
df['customs_description'] = df['customs_description'].astype(str)
df['F_Cat'] = df['F_Cat'].astype(str)
df['quantity'] = df['quantity'].astype(int)

In [58]:
import matplotlib
list_colours = []
for name, hex in matplotlib.colors.cnames.items():
    name = name.title()
    list_colours.append(name)

# list_colours = []
colours_df = pd.read_csv('colours.csv')
colours_df['colours'] = colours_df['colours'].str.strip()
for j in colours_df['colours'].unique().tolist():
    list_colours.append(j)

for i in list_colours:
    i = i.title()

list_colours = list(dict.fromkeys(list_colours))

df['colour'] = df['title'].str.findall(r'(?i)\b(?:{})\b'.format('|'.join(list_colours))).apply(', '.join)

df2 = df['colour'].str.split(',', expand=True)
df['colour'] = df2[0]

In [59]:
orig_df = df.copy()

In [60]:
d1, d2 = '2024-01-01', '2024-01-31'
month_abb = 'Feb'
channel = 'Leather Company'

In [61]:
df = df[(df['date'] >= d1) & (df['date'] <= d2)]
df = df[df['Channel'] == channel]

In [62]:
df = df[(df['customs_description'] == 'Ladies > Leather Jackets')]
df = df[df['model'] != '5 pads set']

temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
temp_refunded = df[df['order_state'] == 'Order Refunded']

df['model'] = df['model'].fillna("(No assigned category)")
df['colour'] = df['colour'].fillna("(No assigned colour)")
df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

temp_ladies = pd.concat([temp_dispatched, temp_refunded], axis=1)
temp_ladies = temp_ladies[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp_ladies['Units Sold'] = temp_ladies['Units Sold'].fillna(0)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].fillna(0)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].fillna(0)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].fillna(0)
temp_ladies['Final Revenue (£)'] = temp_ladies['Total Revenue (£)'] - temp_ladies['Total Refund (£)']
temp_ladies.loc['Total', :] = temp_ladies.sum(numeric_only=True).values
temp_ladies['Units Sold'] = temp_ladies['Units Sold'].astype(int)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].astype(int)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].astype(int)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].astype(int)
temp_ladies['Final Revenue (£)'] = temp_ladies['Final Revenue (£)'].astype(int)

In [63]:
df = orig_df[(orig_df['date'] >= d1) & (orig_df['date'] <= d2)]
df = df[df['Channel'] == channel]

In [64]:
df = df[(df['customs_description'] == 'Mens > Leather Jackets')]

temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
temp_refunded = df[df['order_state'] == 'Order Refunded']

df['model'] = df['model'].fillna("(No assigned category)")
df['colour'] = df['colour'].fillna("(No assigned colour)")
df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

temp = pd.concat([temp_dispatched, temp_refunded], axis=1)
temp = temp[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp['Units Sold'] = temp['Units Sold'].fillna(0)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].fillna(0)
temp['Units Refunded'] = temp['Units Refunded'].fillna(0)
temp['Total Refund (£)'] = temp['Total Refund (£)'].fillna(0)
temp['Final Revenue (£)'] = temp['Total Revenue (£)'] - temp['Total Refund (£)']
temp.loc['Total', :] = temp.sum(numeric_only=True).values
temp['Units Sold'] = temp['Units Sold'].astype(int)
temp['Units Refunded'] = temp['Units Refunded'].astype(int)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].astype(int)
temp['Total Refund (£)'] = temp['Total Refund (£)'].astype(int)
temp['Final Revenue (£)'] = temp['Final Revenue (£)'].astype(int)

In [65]:
temp_ladies

Units Sold  \
customs_description      model       colour  attribute_summary               
Ladies > Leather Jackets 1141        Beige   Size: 14                    1   
                                             Size: 16                    1   
                                             Size: 18                    2   
                                     Black   Size: 10                    1   
                         1676        Black   Size: 18                    1   
...                                                                    ...   
                         licra       Black   Size: 16                    1   
                                             Size: 18                    1   
                         moto bomber Black   Size: 8                     1   
                         nirma       Oxblood Size: 16                    1   
Total                                                                  130   

                                                                Total Revenue (£)  \
customs_description      model       colour  attribute_summary                      
Ladies > Leather Jackets 1141        Beige   Size: 14                         129   
                                             Size: 16                         129   
                                             Size: 18                         258   
                                     Black   Size: 10                         129   
                         1676        Black   Size: 18                         129   
...                                                                           ...   
                         licra       Black   Size: 16                         199   
                                             Size: 18                         199   
                         moto bomber Black   Size: 8                          169   
                         nirma       Oxblood Size: 16                         107   
Total                                                                       29871   

                                                                Units Refunded  \
customs_description      model       colour  attribute_summary                   
Ladies > Leather Jackets 1141        Beige   Size: 14                        1   
                                             Size: 16                        0   
                                             Size: 18                        1   
                                     Black   Size: 10                        0   
                         1676        Black   Size: 18                        1   
...                                                                        ...   
                         licra       Black   Size: 16                        0   
                                             Size: 18                        0   
                         moto bomber Black   Size: 8                         0   
                         nirma       Oxblood Size: 16                        0   
Total                                                                       34   

                                                                Total Refund (£)  \
customs_description      model       colour  attribute_summary                     
Ladies > Leather Jackets 1141        Beige   Size: 14                        129   
                                             Size: 16                          0   
                                             Size: 18                        129   
                                     Black   Size: 10                          0   
                         1676        Black   Size: 18                        129   
...                                                                          ...   
                         licra       Black   Size: 16                          0   
                                             Size: 18                          0   
                         moto bomber

In [66]:
temp

Units Sold  \
customs_description    model        colour attribute_summary               
Mens > Leather Jackets 1895         Black  Size: 3XL                   2   
                                           Size: M                     3   
                                           Size: S                     1   
                                           Size: XS                    1   
                                    Tan    Size: 3XL                   1   
...                                                                  ...   
                       samo         Black  Size: M                     1   
                       stefan biker Grey   Size: XL                    1   
                       super bomber Cognac Size: 2XL                   1   
                       winner       Orange Size: S                     1   
Total                                                                133   

                                                              Total Revenue (£)  \
customs_description    model        colour attribute_summary                      
Mens > Leather Jackets 1895         Black  Size: 3XL                        318   
                                           Size: M                          447   
                                           Size: S                          124   
                                           Size: XS                         149   
                                    Tan    Size: 3XL                        159   
...                                                                         ...   
                       samo         Black  Size: M                          249   
                       stefan biker Grey   Size: XL                         159   
                       super bomber Cognac Size: 2XL                        175   
                       winner       Orange Size: S                          115   
Total                                                                     28210   

                                                              Units Refunded  \
customs_description    model        colour attribute_summary                   
Mens > Leather Jackets 1895         Black  Size: 3XL                       0   
                                           Size: M                         0   
                                           Size: S                         0   
                                           Size: XS                        0   
                                    Tan    Size: 3XL                       0   
...                                                                      ...   
                       samo         Black  Size: M                         0   
                       stefan biker Grey   Size: XL                        0   
                       super bomber Cognac Size: 2XL                       0   
                       winner       Orange Size: S                         0   
Total                                                                     23   

                                                              Total Refund (£)  \
customs_description    model        colour attribute_summary                     
Mens > Leather Jackets 1895         Black  Size: 3XL                         0   
                                           Size: M                           0   
                                           Size: S                           0   
                                           Size: XS                          0   
                                    Tan    Size: 3XL                         0   
...                                                                        ...   
                       samo         Black  Size: M                           0   
                       stefan biker Grey   Size: XL                          0   
                       super bomber Cognac Size: 2XL                         0   
                       winner       Orange Size: S

In [54]:
final = pd.concat([temp_ladies, temp])

In [55]:
final.to_excel(f'{channel} - {month_abb} 2024 - Leather Jackets.xlsx')

In [56]:
# # Leather Company
# import matplotlib.pyplot as plt

# temp_dispatched = temp_dispatched[temp_dispatched['price_inc'] < 400]
# temp_refunded = temp_refunded[temp_refunded['price_inc'] < 400]

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
# fig.suptitle('Leather Company - Ladies Leather Jackets')
# plt.figure(figsize=(20,6))
# bins = [109, 119, 129, 139, 149, 159, 169, 179]
# ax1.hist(temp_dispatched['price_inc'], bins=10, edgecolor="k")
# ax1.set_title('Sold')
# ax2.hist(temp_refunded['price_inc'], bins=10, edgecolor="k")
# ax2.set_title('Refunded')
# y_bins = [0, 10, 20, 30, 40, 50]
# ax1.set_yticks(y_bins)
# ax2.set_yticks(y_bins)
# # ax1.set_xticks(bins)
# # ax2.set_xticks(bins)
# plt.show()